### Twilight Struggle

In [1]:
import numpy as np
import random

from twilight_map import *
from game_mechanics import *
from twilight_cards import *

### Game Instance

In [ ]:
# this is a temporary measure. It should go into UI.new()
g = Game()
g.ui.run()

Initalising game..

----- USSR Player: -----
You may modify 6 influence in these countries. Type in their country indices, separated by commas (no spaces).
17	Poland
21	Romania
7	Finland
20	Yugoslavia
22	Bulgaria
19	Hungary
14	Austria
16	East_Germany
18	Czechoslovakia
> q

----- US Player: -----
You may modify 7 influence in these countries. Type in their country indices, separated by commas (no spaces).
9	Benelux
10	France
6	Sweden
7	Finland
13	Greece
11	Spain_Portugal
8	Denmark
23	Turkey
4	UK
5	Norway
15	West_Germany
14	Austria
12	Italy
3	Canada
> q

----- US Player: -----
You may modify 2 influence in these countries. Type in their country indices, separated by commas (no spaces).
3	Canada
4	UK
26	Israel
30	Iran
42	Australia
44	Philippines
45	Japan
47	South_Korea
59	South_Africa
72	Panama
> q

----- USSR Player: -----
You may headline any of these cards. Type in the card index.
1	Asia_Scoring
2	Europe_Scoring
3	Middle_East_Scoring
4	Duck_and_Cover - 3
5	Five_Year_Plan - 3
7	Socialis